# External Data Retreival

 - ENTSO-E: Wind, Solar, Load forecast and actual, Day-Ahead Price

In [1]:
import requests
from datetime import timedelta, datetime
import pandas as pd
import xml.etree.ElementTree as ET
import re
import numpy as np

In [ ]:
# API function
def entsoe_extract(url):
    payload = {}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)

    if response.status_code == 200:
        root = ET.fromstring(response.content)
        m = re.match(r'\{(.*)\}', root.tag)
        namespace = m.group(1) if m else ' '
        ns = {'ns': namespace}

        data = []

        for timeseries in root.findall('.//ns:TimeSeries', ns):

            start = datetime.strptime(timeseries.find('.//ns:timeInterval/ns:start', ns).text, '%Y-%m-%dT%H:%MZ')
            resolution = timeseries.find('.//ns:resolution', ns).text
            delta = timedelta(minutes=15) if resolution == 'PT15M' else timedelta(hours=1)

            point_dict = {}

            for point in timeseries.findall('.//ns:Point', ns):
                position = int(point.find('ns:position', ns).text)
                timestamp = start + (position - 1) * delta + timedelta(hours=1)

                for child in point:
                    tag = child.tag
                    if '}' in tag:
                        tag = tag.split('}')[1]
                    if tag != 'position':
                        value = float(child.text)
                        point_dict[position] = {'timestamp': timestamp, 'field': tag, 'value': value}

            max_pos = max(point_dict.keys())
            for pos in range(1, max_pos + 1):
                if pos in point_dict:
                    data.append(point_dict[pos])
                else:
                    timestamp = start + (pos - 1) * delta + timedelta(hours=1)
                    data.append({'timestamp': timestamp, 'field': tag, 'value': np.nan})

        df = pd.DataFrame(data)
    else:
        print(f'Failed to retrieve data: {response.status_code}')
        df = pd.DataFrame()

    return df


def entsoe_extract_m(url):
    payload = {}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)

    if response.status_code == 200:
        root = ET.fromstring(response.content)
        m = re.match(r'\{(.*)\}', root.tag)
        namespace = m.group(1) if m else ' '
        ns = {'ns': namespace}

        data = []

        for timeseries in root.findall('.//ns:TimeSeries', ns):
            mrid_text = timeseries.find('ns:mRID', ns).text
            if  mrid_text != '1':
              continue

            start = datetime.strptime(timeseries.find('.//ns:timeInterval/ns:start', ns).text, '%Y-%m-%dT%H:%MZ')
            resolution = timeseries.find('.//ns:resolution', ns).text
            delta = timedelta(minutes=15) if resolution == 'PT15M' else timedelta(hours=1)

            point_dict = {}

            for point in timeseries.findall('.//ns:Point', ns):
                position = int(point.find('ns:position', ns).text)
                timestamp = start + (position - 1) * delta + timedelta(hours=1)

                for child in point:
                    tag = child.tag
                    if '}' in tag:
                        tag = tag.split('}')[1]
                    if tag != 'position':
                        value = float(child.text)
                        point_dict[position] = {'timestamp': timestamp, 'field': tag, 'value': value}

            max_pos = max(point_dict.keys())
            for pos in range(1, max_pos + 1):
                if pos in point_dict:
                    data.append(point_dict[pos])
                else:
                    timestamp = start + (pos - 1) * delta + timedelta(hours=1)
                    data.append({'timestamp': timestamp, 'field': tag, 'value': np.nan})

        df = pd.DataFrame(data)
    else:
        print(f'Failed to retrieve data: {response.status_code}')
        df = pd.DataFrame()

    return df


def fetch_entsoe_range_with_offset(start_date_str, end_date_str):

    start_date = datetime.strptime(start_date_str, "%Y%m%d%H%M")
    end_date = datetime.strptime(end_date_str, "%Y%m%d%H%M")
    max_days = 51
    all_dfs = []

    while start_date < end_date:
        segment_end = min(start_date + timedelta(days=max_days), end_date)
        period_start_str = start_date.strftime("%Y%m%d%H%M")
        period_end_str = segment_end.strftime("%Y%m%d%H%M")

        for offset in range(0, 4900, 100):
            url = f"https://web-api.tp.entsoe.eu/api?securityToken=f7d6cf51-b86d-4a1a-bab9-11ce4cd3fcb6&documentType=A44&out_Domain=10Y1001A1001A82H&in_Domain=10Y1001A1001A82H&periodStart={period_start_str}&periodEnd={period_end_str}&contract_MarketAgreement.type=A01&classificationSequence_AttributeInstanceComponent.position=2&offset={offset}"
            df = entsoe_extract(url)
            if df.empty:
    #            print(f"Offset {offset}: No more data, stopping.")
                break
  #          print(f"Offset {offset}: {len(df)} rows")
            all_dfs.append(df)

        start_date = segment_end  # 推进下一段时间

    day_ahead_price = pd.concat(all_dfs, ignore_index=True)
    day_ahead_price = day_ahead_price.drop_duplicates()
    return day_ahead_price


def fetch_concat(start_date_str, end_date_str, url_t, func):
    all_dfs = []
    start_date = datetime.strptime(start_date_str, "%Y%m%d%H%M")
    end_date = datetime.strptime(end_date_str, "%Y%m%d%H%M")
    max_days = 365

    while start_date < end_date:
        segment_end = min(start_date + timedelta(days=max_days), end_date)
        period_start_str = start_date.strftime("%Y%m%d%H%M")
        period_end_str = segment_end.strftime("%Y%m%d%H%M")

        url = url_t.format(start_date_str=period_start_str, end_date_str=period_end_str)

        print(f"Fetching: {period_start_str} → {period_end_str}")
        df = func(url)
        print(df.shape)
        all_dfs.append(df)

        start_date = segment_end


    data = pd.concat(all_dfs, ignore_index=True)
    data = data.drop_duplicates().reset_index(drop=True)
    print(data.shape)
    return data

In [ ]:
# Load
actual_total_load_url = "https://web-api.tp.entsoe.eu/api?securityToken=f7d6cf51-b86d-4a1a-bab9-11ce4cd3fcb6&documentType=A65&processType=A16&outBiddingZone_Domain=10Y1001A1001A82H&periodStart={start_date_str}&periodEnd={end_date_str}"
day_ahead_total_load_forecast_url = "https://web-api.tp.entsoe.eu/api?securityToken=f7d6cf51-b86d-4a1a-bab9-11ce4cd3fcb6&documentType=A65&processType=A01&outBiddingZone_Domain=10Y1001A1001A82H&periodStart={start_date_str}&periodEnd={end_date_str}"

# Generation Actual
actual_generation_solar_url = "https://web-api.tp.entsoe.eu/api?securityToken=f7d6cf51-b86d-4a1a-bab9-11ce4cd3fcb6&documentType=A74&processType=A16&in_Domain=10Y1001A1001A82H&periodStart={start_date_str}&periodEnd={end_date_str}&PsrType=B16"
actual_generation_wind_offshore_url = "https://web-api.tp.entsoe.eu/api?securityToken=f7d6cf51-b86d-4a1a-bab9-11ce4cd3fcb6&documentType=A74&processType=A16&in_Domain=10Y1001A1001A82H&periodStart={start_date_str}&periodEnd={end_date_str}&PsrType=B18"
actual_generation_wind_onshore_url = "https://web-api.tp.entsoe.eu/api?securityToken=f7d6cf51-b86d-4a1a-bab9-11ce4cd3fcb6&documentType=A74&processType=A16&in_Domain=10Y1001A1001A82H&periodStart={start_date_str}&periodEnd={end_date_str}&PsrType=B19"

# Generation Forecast
day_ahead_generation_solar_forecasts_url = "https://web-api.tp.entsoe.eu/api?securityToken=f7d6cf51-b86d-4a1a-bab9-11ce4cd3fcb6&documentType=A69&processType=A01&in_Domain=10Y1001A1001A82H&periodStart={start_date_str}&periodEnd={end_date_str}&PsrType=B16"
day_ahead_generation_wind_offshore_forecasts_url = "https://web-api.tp.entsoe.eu/api?securityToken=f7d6cf51-b86d-4a1a-bab9-11ce4cd3fcb6&documentType=A69&processType=A01&in_Domain=10Y1001A1001A82H&periodStart={start_date_str}&periodEnd={end_date_str}&PsrType=B18"
day_ahead_generation_wind_onshore_url = "https://web-api.tp.entsoe.eu/api?securityToken=f7d6cf51-b86d-4a1a-bab9-11ce4cd3fcb6&documentType=A69&processType=A01&in_Domain=10Y1001A1001A82H&periodStart={start_date_str}&periodEnd={end_date_str}&PsrType=B19"


In [45]:
load_forecast = fetch_concat(
    start_date_str= "202401010000",  #"202101070000",
    end_date_str="202408220000" ,  #"202312310000",
    url_t = day_ahead_total_load_forecast_url,
    func = entsoe_extract)

Fetching: 202401010000 → 202408220000
(22556, 3)
(22556, 3)


In [46]:
day_ahead_price = fetch_entsoe_range_with_offset(
    start_date_str="202401010000",  #"202101070000",
    end_date_str="202408220000" #"202312310000"
)
print(f"day_ahead_price: {day_ahead_price.shape}")

day_ahead_price: (22556, 3)


In [47]:
generation_solar_forecast = fetch_concat(
    start_date_str= "202401010000", #"202101070000",
    end_date_str="202408220000",#"202312310000",
    url_t = day_ahead_generation_solar_forecasts_url,
    func = entsoe_extract)

Fetching: 202401010000 → 202408220000
(22556, 3)
(22556, 3)


In [48]:
generation_wind_onshore_forecast = fetch_concat(
    start_date_str="202401010000", # "202101070000",
    end_date_str="202408220000", #"202312310000",
    url_t = day_ahead_generation_wind_onshore_url,
    func = entsoe_extract)

Fetching: 202401010000 → 202408220000
(22556, 3)
(22556, 3)


In [49]:
generation_wind_offshore_forecast = fetch_concat(
    start_date_str="202401010000" ,#"202101070000",
    end_date_str="202408220000", #"202312310000",
    url_t = day_ahead_generation_wind_offshore_forecasts_url,
    func = entsoe_extract)

Fetching: 202401010000 → 202408220000
(22556, 3)
(22556, 3)


In [50]:
load_actual = fetch_concat(
    start_date_str="202401010000", #"202101062300",
    end_date_str="202408220000", #"202312310000",
    url_t = actual_total_load_url,
    func = entsoe_extract_m)

Fetching: 202401010000 → 202408220000
(22464, 3)
(22464, 3)


In [51]:
generation_solar_actual = fetch_concat(
    start_date_str="202401010000", #"202101062300",
    end_date_str="202408220000",#"202312310000",
    url_t = actual_generation_solar_url,
    func = entsoe_extract_m)

Fetching: 202401010000 → 202408220000
(22464, 3)
(22464, 3)


In [52]:
generation_wind_offshore_actual = fetch_concat(
    start_date_str="202401010000",#"202101062300",
    end_date_str="202408220000", #"202312310000",
    url_t = actual_generation_wind_offshore_url,
    func = entsoe_extract_m)

Fetching: 202401010000 → 202408220000
(22464, 3)
(22464, 3)


In [53]:
generation_wind_onshore_actual = fetch_concat(
    start_date_str="202401010000",#"202101062300",
    end_date_str="202408220000", #"202312310000",
    url_t = actual_generation_wind_onshore_url,
    func = entsoe_extract_m)

Fetching: 202401010000 → 202408220000
(22464, 3)
(22464, 3)


In [94]:
dfs = {
    'day_ahead_price': day_ahead_price,
    'load_forecast': load_forecast,
    'load_actual': load_actual,
    'generation_solar_actual': generation_solar_actual,
    'generation_wind_onshore_actual': generation_wind_onshore_actual,
    'generation_wind_offshore_actual': generation_wind_offshore_actual,
    'generation_wind_onshore_forecast': generation_wind_onshore_forecast,
    'generation_wind_offshore_forecast': generation_wind_offshore_forecast,
    'generation_solar_forecast': generation_solar_forecast

}

df_wide_list = []
for name, df in dfs.items():
    pivoted_df = df.pivot(index='timestamp', columns='field', values='value')

    pivoted_df.columns = [f"{name}" for col in pivoted_df.columns]
    df_wide_list.append(pivoted_df)


df_final = pd.concat(df_wide_list, axis=1).reset_index()


In [95]:
df_final.isnull().sum()

timestamp                              0
day_ahead_price                      119
load_forecast                          0
load_actual                           92
generation_solar_actual               92
generation_wind_onshore_actual        92
generation_wind_offshore_actual       92
generation_wind_onshore_forecast       0
generation_wind_offshore_forecast      0
generation_solar_forecast              0
dtype: int64

In [ ]:
# For training set
df_variables = df_final[(df_final['timestamp']<='2023-12-25 23:45:00') & (df_final['timestamp']>='2021-01-07 00:00:00') ].reset_index(drop=True)
dst_ranges = [
    (pd.Timestamp('2021-03-28 00:00:00'), pd.Timestamp('2021-10-30 23:59:59')),
    (pd.Timestamp('2022-03-27 00:00:00'), pd.Timestamp('2022-10-29 23:59:59')),
    (pd.Timestamp('2023-03-26 00:00:00'), pd.Timestamp('2023-10-28 23:59:59')),
]

mask = pd.Series(False, index=df_variables.index)
for start, end in dst_ranges:
    mask |= (df_variables['timestamp'] >= start) & (df_variables['timestamp'] <= end)

df_variables.loc[mask, 'timestamp'] = df_variables.loc[mask, 'timestamp'] + pd.Timedelta(hours=1)

df_variables['day_ahead_price'] = df_variables['day_ahead_price'].ffill()

df_variables['timestamp'] = pd.to_datetime(df_variables['timestamp'])

df_variables

,timestamp,day_ahead_price,load_forecast,load_actual,generation_solar_actual,generation_wind_onshore_actual,generation_wind_offshore_actual,generation_wind_onshore_forecast,generation_wind_offshore_forecast,generation_solar_forecast
0,2021-01-07 00:00:00,69.92,48553.0,54269.0,1.0,4682.0,1487.0,5660.0,1602.0,0.0
1,2021-01-07 00:15:00,53.12,47775.0,53619.0,1.0,4676.0,1379.0,5718.0,1570.0,0.0
2,2021-01-07 00:30:00,43.63,47348.0,52866.0,1.0,4734.0,1271.0,5774.0,1532.0,0.0
3,2021-01-07 00:45:00,35.64,47201.0,52446.0,1.0,4836.0,1202.0,5821.0,1498.0,0.0
4,2021-01-07 01:00:00,54.55,46576.0,51882.0,1.0,4870.0,1187.0,5862.0,1675.0,0.0
...,...,...,...,...,...,...,...,...,...,...
103963,2023-12-25 22:45:00,5.04,43088.0,43530.0,3.0,28958.0,6813.0,33823.0,5298.0,0.0
103964,2023-12-25 23:00:00,32.77,42401.0,43107.0,3.0,29217.0,6896.0,34167.0,5272.0,0.0
103965,2023-12-25 23:15:00,15.08,41926.0,42284.0,3.0,29826.0,6832.0,34548.0,5260.0,0.0
103966,2023-12-25 23:30:00,5.07,41316.0,41532.0,3.0,29831.0,6691.0,34919.0,5258.0,0.0


In [ ]:
# test 2024
df_variables = df_final[(df_final['timestamp']<='2024-08-22 23:45:00') & (df_final['timestamp']>='2024-01-01 00:00:00') ].reset_index(drop=True)
dst_ranges = [
    (pd.Timestamp('2024-03-31 00:00:00'), pd.Timestamp('2024-08-21 23:45:00'))
]

mask = pd.Series(False, index=df_variables.index)
for start, end in dst_ranges:
    mask |= (df_variables['timestamp'] >= start) & (df_variables['timestamp'] <= end)

df_variables.loc[mask, 'timestamp'] = df_variables.loc[mask, 'timestamp'] + pd.Timedelta(hours=1)

df_variables['day_ahead_price'] = df_variables['day_ahead_price'].ffill()

df_variables['timestamp'] = pd.to_datetime(df_variables['timestamp'])

df_variables

,timestamp,day_ahead_price,load_forecast,load_actual,generation_solar_actual,generation_wind_onshore_actual,generation_wind_offshore_actual,generation_wind_onshore_forecast,generation_wind_offshore_forecast,generation_solar_forecast
0,2024-01-01 00:00:00,39.91,41204.0,NaN,NaN,NaN,NaN,30322.0,5331.0,0.0
1,2024-01-01 00:15:00,-0.04,40266.0,NaN,NaN,NaN,NaN,30167.0,5284.0,0.0
2,2024-01-01 00:30:00,-9.01,39492.0,NaN,NaN,NaN,NaN,30032.0,5241.0,0.0
3,2024-01-01 00:45:00,-29.91,38965.0,NaN,NaN,NaN,NaN,29935.0,5195.0,0.0
4,2024-01-01 01:00:00,25.28,38678.0,39745.0,3.0,29875.0,5464.0,29893.0,5132.0,0.0
...,...,...,...,...,...,...,...,...,...,...
22551,2024-08-22 21:45:00,69.30,46811.0,NaN,NaN,NaN,NaN,16064.0,4984.0,0.0
22552,2024-08-22 22:00:00,98.76,45854.0,NaN,NaN,NaN,NaN,16305.0,5006.0,0.0
22553,2024-08-22 22:15:00,89.50,45076.0,NaN,NaN,NaN,NaN,16435.0,5031.0,0.0
22554,2024-08-22 22:30:00,66.70,44590.0,NaN,NaN,NaN,NaN,16561.0,5060.0,0.0


In [97]:
df_variables.isnull().sum()

timestamp                             0
day_ahead_price                       0
load_forecast                         0
load_actual                          92
generation_solar_actual              92
generation_wind_onshore_actual       92
generation_wind_offshore_actual      92
generation_wind_onshore_forecast      0
generation_wind_offshore_forecast     0
generation_solar_forecast             0
dtype: int64

In [ ]:
df_variables['timestamp'] = pd.to_datetime(df_variables['timestamp'])
df_variables.to_csv('external_variables_2024.csv', index=False)